In [2]:
#Utilizada no Pipeline
import os
import pandas as pd 

def load_spam_data():
    matrix = []
    spam = open(os.path.join(os.getcwd()+ "\\datasets\\spambase\\", "spambase.data"))
    for row in spam:
        row_nums_string = row.split(',')
        row_nums = [float(i) for i in row_nums_string]
        matrix.append(row_nums)
    return pd.DataFrame(matrix)
spamBase = load_spam_data()
spamBase.head()
print("Ok")

Ok


In [3]:
#spamBase[57]

In [4]:
#Utilizada no Pipeline

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
class StratDataTrain():
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return self.fit_transform
    def fit_transform(self, X, y=None):
        #X[57] = np.ceil(X["median_income"] / 1.5)
        #X["income_cat"].where(X["income_cat"] < 5, 5.0, inplace=True)
        
        split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=43)
        for train_index, test_index in split.split(X, X[57]):
            strat_train_set = X.loc[train_index]
            strat_test_set = X.loc[test_index]
        #for set_ in (strat_train_set, strat_test_set):
            #set_.drop(57, axis=1, inplace=True)
        return strat_train_set.drop(57, axis=1)
    
class StratDataTest():
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return self.fit_transform
    def fit_transform(self, X, y=None):
        temp = X.drop(57, axis=1)
        return temp#.drop("income_cat", axis=1)
        
print("Ok")

Ok


In [39]:
class SpamClassfier():
    def __init__(self):
        self.X = []
        self.Y = []
        self.values = []
    def fit(self, X, y=None):
        self.X = X
        self.Y = y
        
        total = len((self.Y).tolist())
        total_spam=0
        total_not_spam=0
        class_spam = y.tolist()
        for i in range(total):
            if(class_spam[i]==1):
                total_spam += 1
            else:
                total_not_spam +=1
#Prgar 20% iniciais de spam e não spam
        
        for j in range(56):
            cont_spam = 0
            cont_not_spam = 0
            for i in range(total):
                if(class_spam[i]==1):
                    cont_spam+=X[i][j]
                else:
                    cont_not_spam+=X[i][j]
            ps = total_spam/total
            pns = total_not_spam/total
            if(cont_not_spam*(pns-ps)/ps>cont_spam*(pns-ps)/pns):
                self.values.append(0)
            else:
                self.values.append(1)
        
        return self
    def transform(self, X, y=None):
        return self.fit_transform
    def predict(self, x_test):
        total = len(self.X)
        predicts = []
        for i in range(len(x_test)):
            contS = 0
            contNS = 0
            for f in range(len(self.values)):
                if((x_test[i][f]) !=0):
                    if((self.values)[f] == 1):
                        contS +=1
                    else:
                        contNS +=1
            if(contS>contNS):
                predicts.append(1)
            else:
                predicts.append(0)
            
        return predicts
    def fit_transform(self, X, y=None):
        temp = X.drop(57, axis=1)
        return temp#.drop("income_cat", axis=1)
        
print("Ok")

Ok


In [40]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelBinarizer 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

num_pipeline = Pipeline([
    ('strat', StratDataTrain()),
    ('p', PolynomialFeatures()),
    ('std_scaler', MinMaxScaler()),
])

num_pipeline_no_strat = Pipeline([
    ('strat', StratDataTest()),
    ('p', PolynomialFeatures()),
    ('std_scaler', MinMaxScaler()),
])

print("Ok")

Ok


In [41]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
x = []
labels = []
z=[]
for test in range(2):
    method = [GaussianNB(), BernoulliNB(), MultinomialNB(), SpamClassfier()]
    if(test==1):
        print("Treino separado do teste: ")
        print("\n")
    else:
        print("Treino igual ao teste: ")
        print("\n")
    for m in method:
        X = load_spam_data()
        split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=43)
        for train_index, test_index in split.split(X, X[57]):
            strat_train_set = X.loc[train_index]
            strat_test_set = X.loc[test_index]
        strat_train_set_labels=strat_train_set[57].copy()

        spamX = num_pipeline.fit_transform(load_spam_data())
        spamY = strat_train_set_labels
        if(test==1):
            spam_testY = strat_test_set[57].copy()
            spam_testX = num_pipeline_no_strat.fit_transform(strat_test_set)
        else:
            spam_testX = spamX
            spam_testY = spamY


        lin_reg = m
        lin_reg.fit(spamX, spamY)
        spam_predictions = lin_reg.predict(spam_testX)

        errosSpam = 0
        acertosSpam = 0
        errosNotSpam = 0
        acertosNotSpam = 0
        total = 0
        u1 = spam_predictions#.tolist()
        u2 = spam_testY.tolist()
        for i in range(len(u1)):
            total = total + 1
            if(u1[i]!=u2[i]):
                if(u2[i]==0):
                    errosNotSpam +=1
                else:
                    errosSpam +=1
            else:
                if(u2[i]==0):
                    acertosNotSpam +=1
                else:
                    acertosSpam +=1
        
        print("method: ",lin_reg)
        print("total erros Spam:", errosSpam)
        print("total acertos Spam:", acertosSpam)
        print("total erros não Spam:", errosNotSpam)
        print("total acertos não Spam:", acertosNotSpam)
        print("total", total)
        print("taxa de acertos Spam: {:.2f}".format((acertosSpam*100)/(acertosSpam + errosSpam)), "%")
        print("taxa de acertos não Spam: {:.2f}".format((acertosNotSpam*100)/(acertosNotSpam + errosNotSpam)), "%")
        print("taxa de acertos geral: {:.2f}".format(((acertosSpam + acertosNotSpam)*100)/(total)), "%")
        print("\n")

        

Treino igual ao teste: 


method:  GaussianNB(priors=None)
total erros Spam: 14
total acertos Spam: 1436
total erros não Spam: 563
total acertos não Spam: 1667
total 3680
taxa de acertos Spam: 99.03 %
taxa de acertos não Spam: 74.75 %
taxa de acertos geral: 84.32 %


method:  BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
total erros Spam: 383
total acertos Spam: 1067
total erros não Spam: 103
total acertos não Spam: 2127
total 3680
taxa de acertos Spam: 73.59 %
taxa de acertos não Spam: 95.38 %
taxa de acertos geral: 86.79 %


method:  MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
total erros Spam: 114
total acertos Spam: 1336
total erros não Spam: 229
total acertos não Spam: 2001
total 3680
taxa de acertos Spam: 92.14 %
taxa de acertos não Spam: 89.73 %
taxa de acertos geral: 90.68 %


method:  <__main__.SpamClassfier object at 0x00000230E2FC33C8>
total erros Spam: 206
total acertos Spam: 1244
total erros não Spam: 243
total acertos não Spam: 1987